In [33]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import time
import datetime
import os
import sklearn
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from collections import OrderedDict
from datetime import datetime, timedelta
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

### Load data

In [2]:
data = joblib.load('feature_308.pkl')
data.shape

(161511, 308)

In [3]:
data.head()

,申请编号,贷款类型,信用额度_x,贷款年金,商品价格_x,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,...,分期付款实际支付时间_std,本期贷款金额_mean,本期贷款金额_min,本期贷款金额_max,本期贷款金额_std,本期还款金额_mean,本期还款金额_min,本期还款金额_max,本期还款金额_std,标签
0,0,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,-3292,...,86.838657,1098.133333,1072.0,1100.0,7.229569,880.266667,12.0,1100.0,409.090012,1.0
1,1,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,-826,...,145.851557,803.520000,635.0,976.0,169.155776,778.520000,242.0,976.0,213.114429,0.0
2,2,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,-171,...,165.903744,5578.146341,2574.0,39611.0,5754.484132,5455.414634,607.0,39611.0,5808.108436,0.0
3,3,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,-2916,...,87.467397,4911.500000,4906.0,4912.0,1.732051,4911.500000,4906.0,4912.0,1.732051,0.0
4,5,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,-3588,...,950.202378,1957.629630,33.0,17350.0,3668.444212,1923.222222,33.0,17350.0,3682.385205,0.0


In [4]:
data['标签'] = data['标签'].fillna(-1)
print(data['标签'].value_counts())

 0.0    119000
-1.0     21511
 1.0     21000
Name: 标签, dtype: int64


In [1]:
# from sklearn.utils import shuffle
# data = shuffle(data)
# data = data.iloc[:10000,]
# train_data = data[data['标签']!=-1]
# test_data = data[data['标签']==-1].drop(columns = ['标签'])
# print('train_data：',train_data.shape,'\ntest_data：',test_data.shape)

# x = train_data.drop(columns = '标签')
# y = train_data['标签']
# print('train_x：',x.shape,'\ntrain_label：',y.shape)

# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=1024)

In [5]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.iloc[:50000,]
train_data = data[data['标签']!=-1].drop(columns='申请编号',axis=1)
test_data = data[data['标签']==-1].drop(columns = ['申请编号','标签'])
print('train_data：',train_data.shape,'\ntest_data：',test_data.shape)

x = train_data.drop(columns = '标签')
y = train_data['标签']
print('train_x：',x.shape,'\ntrain_label：',y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=1024)

train_data： (43397, 307) 
test_data： (6603, 306)
train_x： (43397, 306) 
train_label： (43397,)


In [6]:
x_train.columns

Index(['贷款类型', '信用额度_x', '贷款年金', '商品价格_x', '陪同申请人', '出生日期距申请日期天数',
       '工作日期距申请日期天数', '注册日期距申请日期天数', '身份认证日期距申请日期天数', '是否提供手机号',
       ...
       '分期付款实际支付时间_max', '分期付款实际支付时间_std', '本期贷款金额_mean', '本期贷款金额_min',
       '本期贷款金额_max', '本期贷款金额_std', '本期还款金额_mean', '本期还款金额_min', '本期还款金额_max',
       '本期还款金额_std'],
      dtype='object', length=306)

In [6]:
# def ks_min(preds,dtrain):
#         #自定义损失函数
#         labels = dtrain.get_label()
#         fpr,tpr,thresholds= sklearn.metrics.roc_curve(labels,preds,pos_label=1)
#         #ks_1 = 1-max(tpr - fpr)
#         return '1-KS',1-abs(fpr - tpr).max()
    
# def xgb_auto_gs(x_train,y_train,x_test):
#     '''
#     x_train:训练用数据集 dataframe
#     y_train:训练用标签 Series
#     x_test:测试集
    
#     return：
#     result['best_params']:最优参数组
#     result['y_test']:x_test预测结果
#     '''
#     params = OrderedDict()
#     #调参范围(可调整，如需增加调整的参数，可继续在params组中添加)
#     params['tree'] = {'max_depth': [3,5,7,9],'n_estimators': [500,1000,2000]} 
#     params['over_fitting'] = {'gamma':[0,0.1,0.3,0.5],'min_child_weight': [1,4,7,10,15]}
#     params['variable'] = {'subsample':[0.6,0.7,0.8,0.9],'colsample_bytree':[0.6,0.7,0.8,0.9]}
#     #默认参数(可调整)
#     best_params = {    'learning_rate':0.1,            #学习率                          大：欠拟合；小 ：过拟合
#                           'n_estimators':500,          # 树的个数--1000棵树建立xgboost  小：欠拟合；大 ：过拟合 
#                           'max_depth':5,               # 树的深度                       小：欠拟合；大 ：过拟合                 
#                           'min_child_weight':7,      # 叶子节点最小权重               大：欠拟合；小 ：过拟合    
#                           'gamma':0.5,                 # 惩罚项中叶子结点个数前的参数   大：欠拟合；小 ：过拟合    
#                           'subsample':0.7,             # 随机选择%s样本建立决策树       小：欠拟合；大 ：过拟合 
#                           'colsample_bytree':0.7,      # 随机选择%s特征建立决策树       小：欠拟合；大 ：过拟合   
#                           'objective':'binary:logistic','silent':1}
    
#     #分批调试
#     for i in params:
#         print('-----searching params: %s-----' % params[i])
#         xgbc_best = XGBClassifier(**best_params)
#         gs = GridSearchCV(xgbc_best, params[i], n_jobs=-1, cv=5, verbose=1)
#         gs.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric=ks_min, early_stopping_rounds=50,verbose=True)
#         best_parami = gs.best_params_#最优参数
#         print('-----best params: %s-----' % best_parami)
#         for param_i in best_parami:
#             best_params[param_i] = best_parami[param_i]#更新最优参数
#     xgbc_best = XGBClassifier(**best_params)
#     xgbc_best.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric=ks_min, early_stopping_rounds=50,verbose=True)
#     y_pre = xgbc_best.predict_proba(x_test)
#     y_pre = y_pre[:,1]
#     result={'best_params':best_params,'y_test':y_pre}
#     return result

In [7]:
def ks_min(preds,dtrain):
        #自定义损失函数
        labels = dtrain.get_label()
        fpr,tpr,thresholds= sklearn.metrics.roc_curve(labels,preds,pos_label=1)
        #ks_1 = 1-max(tpr - fpr)
        return '1-KS',1-abs(fpr - tpr).max()
    
def xgb_auto_gs(x_train,y_train,x_test):
    '''
    x_train:训练用数据集 dataframe
    y_train:训练用标签 Series
    x_test:测试集
    
    return：
    result['best_params']:最优参数组
    result['y_test']:x_test预测结果
    '''
#     params = OrderedDict()
#     #调参范围(可调整，如需增加调整的参数，可继续在params组中添加)
#     params['tree'] = {'max_depth': [3,5,7,9],'n_estimators': [500,1000,2000]} 
#     params['over_fitting'] = {'gamma':[0,0.1,0.3,0.5],'min_child_weight': [1,4,7,10,15]}
#     params['variable'] = {'subsample':[0.6,0.7,0.8,0.9],'colsample_bytree':[0.6,0.7,0.8,0.9]}
    #默认参数(可调整)
    best_params = {    'learning_rate':0.1,            #学习率                          大：欠拟合；小 ：过拟合
                          'n_estimators':500,          # 树的个数--1000棵树建立xgboost  小：欠拟合；大 ：过拟合 
                          'max_depth':5,               # 树的深度                       小：欠拟合；大 ：过拟合                 
                          'min_child_weight':7,      # 叶子节点最小权重               大：欠拟合；小 ：过拟合    
                          'gamma':0.5,                 # 惩罚项中叶子结点个数前的参数   大：欠拟合；小 ：过拟合    
                          'subsample':0.7,             # 随机选择%s样本建立决策树       小：欠拟合；大 ：过拟合 
                          'colsample_bytree':0.7,      # 随机选择%s特征建立决策树       小：欠拟合；大 ：过拟合   
                          'objective':'binary:logistic','silent':1}
    
#     #分批调试
#     for i in params:
#         print('-----searching params: %s-----' % params[i])
#         xgbc_best = XGBClassifier(**best_params)
#         gs = GridSearchCV(xgbc_best, params[i], n_jobs=-1, cv=5, verbose=1)
#         gs.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric=ks_min, early_stopping_rounds=50,verbose=True)
#         best_parami = gs.best_params_#最优参数
#         print('-----best params: %s-----' % best_parami)
#         for param_i in best_parami:
#             best_params[param_i] = best_parami[param_i]#更新最优参数
    xgbc_best = XGBClassifier(**best_params)
    xgbc_best.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric=ks_min,\
                  early_stopping_rounds=100,verbose=50)
    y_pre = xgbc_best.predict_proba(x_test)
    y_pre = y_pre[:,1]
    result={'best_params':best_params,'y_test':y_pre}
    return result

In [8]:
xgb_auto_gs(x_train,y_train,x_test)

[0]	validation_0-error:0.150398	validation_0-1-KS:0.769127
Multiple eval metrics have been passed: 'validation_0-1-KS' will be used for early stopping.

Will train until validation_0-1-KS hasn't improved in 100 rounds.
[50]	validation_0-error:0.145666	validation_0-1-KS:0.530606
[100]	validation_0-error:0.136172	validation_0-1-KS:0.461651
[150]	validation_0-error:0.130058	validation_0-1-KS:0.402124
[200]	validation_0-error:0.123206	validation_0-1-KS:0.355913
[250]	validation_0-error:0.115556	validation_0-1-KS:0.318414
[300]	validation_0-error:0.109227	validation_0-1-KS:0.280806
[350]	validation_0-error:0.10345	validation_0-1-KS:0.246967
[400]	validation_0-error:0.096199	validation_0-1-KS:0.222651
[450]	validation_0-error:0.090147	validation_0-1-KS:0.202508
[499]	validation_0-error:0.084831	validation_0-1-KS:0.18224


{'best_params': {'colsample_bytree': 0.7,
  'gamma': 0.5,
  'learning_rate': 0.1,
  'max_depth': 5,
  'min_child_weight': 7,
  'n_estimators': 500,
  'objective': 'binary:logistic',
  'silent': 1,
  'subsample': 0.7},
 'y_test': array([0.3938505 , 0.05657872, 0.42592585, ..., 0.2014308 , 0.05194579,
        0.04570697], dtype=float32)}

In [10]:
best_params = { 'learning_rate':0.1,            #学习率                          大：欠拟合；小 ：过拟合
                          'n_estimators':500,          # 树的个数--1000棵树建立xgboost  小：欠拟合；大 ：过拟合 
                          'max_depth':5,               # 树的深度                       小：欠拟合；大 ：过拟合                 
                          'min_child_weight':7,      # 叶子节点最小权重               大：欠拟合；小 ：过拟合    
                          'gamma':0.5,                 # 惩罚项中叶子结点个数前的参数   大：欠拟合；小 ：过拟合    
                          'subsample':0.7,             # 随机选择%s样本建立决策树       小：欠拟合；大 ：过拟合 
                          'colsample_bytree':0.7,      # 随机选择%s特征建立决策树       小：欠拟合；大 ：过拟合   
                          'objective':'binary:logistic','silent':1}
xgbc_best = XGBClassifier(**best_params)
xgbc_best.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric=ks_min,\
                  early_stopping_rounds=100,verbose=50)

[0]	validation_0-error:0.150398	validation_0-1-KS:0.769127
Multiple eval metrics have been passed: 'validation_0-1-KS' will be used for early stopping.

Will train until validation_0-1-KS hasn't improved in 100 rounds.
[50]	validation_0-error:0.145666	validation_0-1-KS:0.530606
[100]	validation_0-error:0.136172	validation_0-1-KS:0.461651
[150]	validation_0-error:0.130058	validation_0-1-KS:0.402124
[200]	validation_0-error:0.123206	validation_0-1-KS:0.355913
[250]	validation_0-error:0.115556	validation_0-1-KS:0.318414
[300]	validation_0-error:0.109227	validation_0-1-KS:0.280806
[350]	validation_0-error:0.10345	validation_0-1-KS:0.246967
[400]	validation_0-error:0.096199	validation_0-1-KS:0.222651
[450]	validation_0-error:0.090147	validation_0-1-KS:0.202508
[499]	validation_0-error:0.084831	validation_0-1-KS:0.18224


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=7, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.7, verbosity=1)

In [12]:
y_pre = xgbc_best.predict_proba(x_test)
y_pre = y_pre[:,1]
fpr,tpr,thresholds= sklearn.metrics.roc_curve(y_test,y_pre,pos_label=1)
max(tpr - fpr)

0.3347164457357572

In [13]:
y_pre = xgbc_best.predict_proba(x_train)
y_pre = y_pre[:,1]
fpr,tpr,thresholds= sklearn.metrics.roc_curve(y_train,y_pre,pos_label=1)
max(tpr - fpr)

0.8177600909596058

In [31]:
pd.set_option('display.max_rows',400)
pd.DataFrame(data.columns)

,0
0,申请编号
1,贷款类型
2,信用额度_x
3,贷款年金
4,商品价格_x
5,陪同申请人
6,出生日期距申请日期天数
7,工作日期距申请日期天数
8,注册日期距申请日期天数
9,身份认证日期距申请日期天数


In [32]:
data.iloc[:20,129:140]

,申请周内日_0,申请周内日_1,申请周内日_3,申请周内日_4,申请周内日_5,申请周内日_6,申请周内日_7,申请时点_0,申请时点_1,申请时点_2,申请时点_3
115393,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
46512,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56502,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
102953,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144327,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120260,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0
69301,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63240,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
135890,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
def k_fold_xgb_train(feature_num):

    feature_importance = joblib.load("feture_importance_data_B")
    features = list(feature_importance["names"])[0:feature_num]
    train_x_1 = train_x[features]
    test_x_1 = test_x[features]
    predictions_xgb = xgb_model(1, train_x_1.values, train_y.values, test_x_1.values, "xgb_model0927_1", "log1")
    sub = pd.DataFrame(columns=["客户号", "违约概率"])
    sub["申请编号"] = list(test_id)
    sub["违约概率"] = list(predictions_lgb)
    sub.to_csv("submit_xgb0927_1.csv", sep=",", index=False, header=0)
    return predictions_xgb, sub